In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from src.hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'bayanpy', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [2]:
interactions_by_chapter = pickle.load(open("data/interactions_by_chapter.pkl", "rb"))

In [3]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [4]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [5]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [6]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,6.182915e-01,0.307398,0.666667,0.715120,0.204660,2.015876e-01,2.015876e-01,78,3700
Ronald Weasley,5.118182e-01,0.110605,0.435897,0.564150,0.101425,1.668781e-01,1.668781e-01,51,1960
Hermione Granger,4.422707e-01,0.064473,0.307692,0.518097,0.076311,1.442010e-01,1.442010e-01,36,1545
Rubeus Hagrid,2.886673e-01,0.068014,0.307692,0.531660,0.052184,9.411978e-02,9.411978e-02,36,1040
Severus Snape,1.490132e-01,0.029914,0.222222,0.488207,0.029499,4.858546e-02,4.858546e-02,26,594
...,...,...,...,...,...,...,...,...,...
Emeric the Evil,8.447688e-06,0.000000,0.017094,0.310541,0.002367,2.754316e-06,2.754316e-06,2,2
Uric the Oddball,8.447688e-06,0.000000,0.017094,0.310541,0.002367,2.754316e-06,2.754316e-06,2,2
Morag MacDougal,6.093707e-05,0.000000,0.008547,0.321351,0.001321,1.986806e-05,1.986806e-05,1,1


In [7]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [8]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [9]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [10]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [11]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [25]:
metrics_df.to_csv("data/metrics.csv")

In [12]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [28]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [33]:
nx.write_gexf(G, "data/graph.gexf")